In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 

In [ ]:
import pandas as pd

pars = load_parameters()

input_file = pars.get('input')
output_file = pars.get('output')

input_col = pars.get('input_col')
output_col = pars.get('output_col')

debug = pars.get('debug')

In [14]:
#import pandas as pd
#input_file = '../../data/stack-overflow/pandas-preprocessedcode-dataset-part2'
#input_col = 'PreprocessedCode2'
#output_file = '../../data/stack-overflow/pandas-preprocessedcode-dataset-part2_1'
#output_col = 'PreprocessedCode2_1'
#debug = True

In [5]:
dataset = pd.read_pickle(input_file)

#### Remove comments

Since our tagged solution don't have the comments, comments should be removed to avoid negative matching

Solution code 
cols = seq[:]

will not match with below code which is the code fragment on stack overflow

cols = seq[:] # copy so we don't mutate seq

INPUT


IN :def set_column_sequence(dataframe, seq, front=True):
   
   '''Takes a dataframe and a subsequence of its columns,    <br>
       returns dataframe with seq as first columns if ""front"" is True, <br>
       and seq as last columns if ""front"" is False. <br>
    '''
    
    cols = seq[:] # copy so we don't mutate seq
    for x in dataframe.columns:
        if x not in cols:
            if front: #we want ""seq"" to be in the front
                #so append current column to the end of the list
                cols.append(x)
            else:
                #we want ""seq"" to be last, so insert this
                #column in the front of the new column list
                #""cols"" we are building:
                cols.insert(0, x)
return dataframe[cols]


Output


IN :def set_column_sequence(dataframe, seq, front=True):
    
    cols = seq[:] 
    for x in dataframe.columns:
        if x not in cols:
            if front:
 
                cols.append(x)
            else:
                
                cols.insert(0, x)
return dataframe[cols]


#### Remove In Terminal

We earlier removed In[] kind of code, but sometimes users use 'IN :'. We would like to remove

Input


IN :def set_column_sequence(dataframe, seq, front=True):
    
    cols = seq[:] 
    for x in dataframe.columns:
        if x not in cols:
            if front:
 
                cols.append(x)
            else:
                
                cols.insert(0, x)
return dataframe[cols]


Output

def set_column_sequence(dataframe, seq, front=True):
    
    cols = seq[:] 
    for x in dataframe.columns:
        if x not in cols:
            if front:
 
                cols.append(x)
            else:
                
                cols.insert(0, x)
return dataframe[cols]


#### Remove Blanks

After doing this, we have some blank lines that should be removed.

Input

def set_column_sequence(dataframe, seq, front=True):
    
    cols = seq[:] 
    for x in dataframe.columns:
        if x not in cols:
            if front:
 
                cols.append(x)
            else:
                
                cols.insert(0, x)
return dataframe[cols]


Output

def set_column_sequence(dataframe, seq, front=True): <br>
    
    cols = seq[:]  
    for x in dataframe.columns: 
        if x not in cols: 
            if front: 
                cols.append(x) 
            else: 
                cols.insert(0, x) 
return dataframe[cols]






In [6]:
import re

def removeBlankLines(content):
    lines = [line for line in content.split(os.linesep) if line.strip() != '']
    return os.linesep.join(lines)

def removeComments(content):
    content = re.sub(re.compile("\'\'\'.*?\'\'\'",re.DOTALL ) ,"" ,content) # remove all occurance streamed comments ('''COMMENT''') from string
    content = re.sub(re.compile("#.*?\n" ) ,"\n" ,content) # remove all occurance singleline comments (#COMMENT\n ) from string
    return content

def removeInTerminal(content):
    content = re.sub('IN.*?:', "", content)
    return content

In [19]:
def preprocessPost(post):
    code = str(post[input_col])
    code = removeComments(code)
    code = removeInTerminal(code)
    code = removeBlankLines(code)
    post[output_col] =  code
    return post

processed_dataset = dataset.apply(preprocessPost, axis=1)

In [20]:
processed_dataset.to_pickle(output_file)

if debug:
    processed_dataset.to_csv(output_file + ".csv", encoding='ISO-8859-1', sep=",", doublequote=True, index=False)